## Appreciation Content Evaluator

#### The program is calling a tensorflow predefine model, and takes appreciation (feedback) email content as input. It evaluate the content and score the various HR checkpoints list in percentage.

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

In [3]:
import tensorflow_hub as hub

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")
embeddings = embed([
    "The quick brown fox jumps over the lazy dog.",
    "I am a sentence for which I would like to get its embedding"])

print(embeddings)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Tensor("StatefulPartitionedCall_1:0", shape=(?, 512), dtype=float32)


In [5]:
embeddings.min()

In [0]:
import numpy as np
import pandas as pd
import ssl
import tensorflow as tf
import tensorflow_hub as hub

from sklearn.preprocessing import MultiLabelBinarizer

citdata=pd.read_csv('/content/gdrive/My Drive/uniform_data.csv')
citations=citdata['Content']
quals=citdata['Label']
context=ssl._create_unverified_context
# print(citdata)

cit_embeddings=hub.text_embedding_column(
    'citations',
    module_spec='http://tfhub.dev/google/universal-sentence-encoder/2'

)
# print(quals)
encoder=MultiLabelBinarizer()
encoder.fit_transform([(x,) for x in quals])
train_encoded=encoder.transform([(x,) for x in quals])
num_quals=len(encoder.classes_)
# print(encoder.classes_)

multi_label_head=tf.contrib.estimator.multi_label_head(
    num_quals,
    loss_reduction=tf.losses.Reduction.SUM_OVER_BATCH_SIZE
)

estimator=tf.contrib.estimator.DNNEstimator(
    head=multi_label_head,
    hidden_units=[256,10],
    feature_columns=[cit_embeddings]
)
features={'citations':np.array(citations)}
labels=np.array(train_encoded)

train_input_fn=tf.estimator.inputs.numpy_input_fn(features,labels,shuffle=True,\
                                                 batch_size=16,num_epochs=1000)
estimator.train(input_fn=train_input_fn)

# print('done')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpjoqcwfok', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7faa4f9974e0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Sa

In [0]:
raw_test=[
    'thank you for always supporting and guiding me whenever i needed you are a great leader and i am happy to be a part of your team'
]
predict_input_fn=tf.estimator.inputs.numpy_input_fn({'citations':np.array(raw_test).astype(np.str)},shuffle=False)
results=estimator.predict(predict_input_fn)
for praise in results:
  top_2=praise['probabilities'].argsort()[-5:][::-1]
  for pred in top_2:
    text_pred=encoder.classes_[pred]
    print(text_pred+':'+str(round(praise['probabilities'][pred]*100,2)) + '%')
  

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpjoqcwfok/model.ckpt-25438
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
teamwork and leadership:66.1%
helping and caring:45.67%
accessible:12.59%
work ethics(accountability_responsibility_credibility_intgrity_fairness):6.64%
manager appreciation:3.02%


In [0]:
print(encoder.classes_)

['accessible' 'anniversary' 'bold and experimentalist' 'customer oriented'
 'general appreciation' 'helping and caring' 'innovative and talented'
 'knowledge holder and seeker' 'manager appreciation'
 'passionate and ambitious' 'result-oriented' 'sense of urgency'
 'teamwork and leadership'
 'work ethics(accountability_responsibility_credibility_intgrity_fairness)']
